In [165]:
import numpy as np
from pylab import hold,show,plot
from scipy import sin,rand,arange
from pybrain.datasets import SequenceClassificationDataSet
from pybrain.structure.modules import LSTMLayer, SoftmaxLayer
from pybrain.supervised import RPropMinusTrainer
from pybrain.tools.validation import testOnSequenceData
from pybrain.tools.shortcuts import buildNetwork


In [166]:
x = (np.arange(10)/float(10))* 20
print x

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.]


In [167]:
y1 = np.sin(x+rand(1)*3.)
print y1

[ 0.52933012  0.55118339 -0.98807656  0.27118649  0.76236977 -0.90570202
 -0.00855971  0.91282621 -0.75117977 -0.28762404]


In [168]:
y2=np.sin((x/2)+rand(1)*3.)
print y2

[ 0.76279204  0.95627036  0.27055812 -0.66390401 -0.98797585 -0.40370726
  0.55172793  0.999907    0.52877619 -0.42850901]


In [169]:
DS = SequenceClassificationDataSet(1,1)

In [21]:
def generateNoisySines( npoints, nseq, noise=0.3 ):
    """ construct a 2-class dataset out of noisy sines
    npoints is the length of each sequence 
    nseq is the number of sequences"""
    x = np.arange(npoints)/float(npoints) * 20.
    y1 = np.sin(x+rand(1)*3.)
    y2 = np.sin(x/2.+rand(1)*3.)
    DS = SequenceClassificationDataSet(1,1, nb_classes=2)
    for _ in range(nseq):
        DS.newSequence()
        buf = rand(npoints)*noise + y1 + (rand(1)-0.5)*noise
        for i in range(npoints):
            DS.addSample([buf[i]],[0])
        DS.newSequence()
        buf = rand(npoints)*noise + y2 + (rand(1)-0.5)*noise
        for i in range(npoints):
            DS.addSample([buf[i]],[1])
    return DS

In [23]:
a=generateNoisySines(10, 1)

In [33]:
range(a.getNumSequences())

[0, 1]

In [40]:
for i in [0,1]:
    for input, target in a.getSequenceIterator(i):
        plot(input,target,"o")
        hold(True)
show()
        

In [44]:
range(1)

[0]

In [47]:
buf = rand(10) + y1 + (rand(1)-0.5)
print buf

[ 1.6600115   0.5816573  -0.91569517  1.39916126 -0.0111351  -1.00316249
  0.59948874  0.99714332 -0.50764733  0.80392462]


In [50]:
print rand(10)

[ 0.73301411  0.17152669  0.74448908  0.85464726  0.58327779  0.01939511
  0.98540952  0.81088825  0.83646127  0.26049625]


In [133]:
plot(x,y1,'-o')
plot(x,y2,'-*')
show()

In [170]:
for i in y1:
    DS.addSample([i],[0])

In [171]:
DS.newSequence()

In [172]:
for i in y2:
    DS.addSample([i],[1])

In [173]:
DS.getNumSequences()

2

In [174]:
DS.getSequence(0)

[array([[ 0.52933012],
        [ 0.55118339],
        [-0.98807656],
        [ 0.27118649],
        [ 0.76236977],
        [-0.90570202],
        [-0.00855971],
        [ 0.91282621],
        [-0.75117977],
        [-0.28762404]]), array([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])]

In [139]:
y1

array([ 0.99999016, -0.41210874, -0.65699466,  0.95892124, -0.14110942,
       -0.84147677,  0.8414652 ,  0.1411306 , -0.95892731,  0.65697853])

In [140]:
DS._convertToOneOfMany(bounds=[0.,1.] )
# Adds class to DS.

In [175]:
print "Number of training patterns: ", len(DS)
print "Input and output dimensions: ", DS.indim, DS.outdim
print "First sample (input, target, class):"
print DS['input'][0], DS['target'][0], DS['class'][0]


Number of training patterns:  20
Input and output dimensions:  1 1
First sample (input, target, class):
[ 0.52933012] [0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [142]:
DS['class']

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [176]:
rnn = buildNetwork( DS.indim, 5, DS.outdim, hiddenclass=LSTMLayer, outclass=SoftmaxLayer, outputbias=False, recurrent=True)

In [177]:
rnn.params

array([ -9.91905745e-01,   5.64168933e-02,  -2.98493784e-01,
         6.74733038e-01,  -8.80597528e-01,  -5.79276201e-01,
        -1.10332326e+00,  -2.09545542e+00,   8.59509664e-01,
         5.99675409e-01,   1.69810821e+00,   4.02710620e-01,
         1.25488669e+00,   6.11877213e-03,   2.64757386e-01,
        -1.86422616e+00,   1.10034524e+00,  -8.40389208e-01,
        -3.61277911e-01,   8.36011093e-01,   4.94318769e-01,
         5.10736588e-01,   1.16329234e+00,  -1.19291508e+00,
         7.80909028e-01,   1.90848404e-01,  -8.54794465e-01,
        -2.70814169e-02,  -4.24256627e-02,  -6.94372504e-01,
         1.63872937e+00,   2.33585351e-01,   1.46664451e+00,
         7.31247465e-01,   5.15711715e-01,   1.14907327e+00,
        -5.88850062e-01,   1.49962993e-02,  -1.91204896e+00,
        -7.06557781e-01,   6.41081982e-01,  -3.43359825e-01,
        -1.38762953e+00,  -1.66416590e-01,   4.68063019e-01,
        -1.03974431e+00,  -2.02205392e+00,   4.11616249e-01,
        -1.41440351e-01,

In [178]:
trainer = RPropMinusTrainer( rnn, dataset=DS, verbose=True )

In [179]:
rnn.sortModules()

In [180]:
rnn.activate(.34)

array([ 1.])

In [181]:
for i in range(100):
    trainer.trainEpochs( 2 )
    trnresult = 100. * (1.0-testOnSequenceData(rnn, DS))
    print("train error: %5.2f%%" % trnresult)



epoch      0  total error         0.25   avg weight       0.93991
epoch      1  total error         0.25   avg weight       0.94591
train error:  0.00%
epoch      2  total error         0.25   avg weight       0.96548
epoch      3  total error         0.25   avg weight       0.98951
train error:  0.00%
epoch      4  total error         0.25   avg weight          1.01
epoch      5  total error         0.25   avg weight        1.0543
train error:  0.00%
epoch      6  total error         0.25   avg weight        1.1264
epoch      7  total error         0.25   avg weight        1.2298
train error:  0.00%
epoch      8  total error         0.25   avg weight        1.3657
epoch      9  total error         0.25   avg weight        1.5538
train error:  0.00%
epoch     10  total error         0.25   avg weight         1.801
epoch     11  total error         0.25   avg weight        2.1173
train error:  0.00%
epoch     12  total error         0.25   avg weight        2.5144
epoch     13  total er

In [182]:
print DS['input']
print DS['target']

[[ 0.52933012]
 [ 0.55118339]
 [-0.98807656]
 [ 0.27118649]
 [ 0.76236977]
 [-0.90570202]
 [-0.00855971]
 [ 0.91282621]
 [-0.75117977]
 [-0.28762404]
 [ 0.76279204]
 [ 0.95627036]
 [ 0.27055812]
 [-0.66390401]
 [-0.98797585]
 [-0.40370726]
 [ 0.55172793]
 [ 0.999907  ]
 [ 0.52877619]
 [-0.42850901]]
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [152]:
rnn.activate(.34)

array([ 0.,  1.])

In [153]:
rnn.reset()

In [154]:
testOnSequenceData(rnn,DS)

1.0

In [161]:
for i in y1:
    print rnn.activate(i)
    rnn.reset()

[  1.00000000e+00   8.01601307e-21]
[ 0.  1.]
[ 0.  1.]
[ 0.5  0.5]
[ 0.  1.]
[ 0.  1.]
[ 0.5  0.5]
[  4.57148581e-257   1.00000000e+000]
[ 0.  1.]
[ 0.5  0.5]


In [162]:
for i in y2:
    print rnn.activate(i)
    rnn.reset()

[  1.32787954e-17   1.00000000e+00]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[  2.19476720e-198   1.00000000e+000]
[ 0.5  0.5]
[ 0.5  0.5]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]


In [164]:
rnn.params

array([  4.33475700e+01,   5.57011843e+01,   3.54452828e+01,
         3.27501197e+01,   3.45615292e+01,   5.24702771e+01,
         5.88004545e+01,   3.00480303e+00,   1.82491194e-01,
         1.49663086e+00,   7.25987381e-01,  -9.62709513e-01,
         2.55862736e+01,  -4.36679779e+00,   1.26840073e+01,
         5.04582502e+01,   4.91380753e+01,   3.70005479e+01,
         3.91910410e+01,   1.60686729e+01,   2.09007012e-01,
         5.62077189e+00,   7.80029032e+00,   1.21259538e+01,
         7.33058158e+00,  -1.04414895e+01,  -3.11113597e+00,
        -4.38228266e+00,  -2.62485406e+00,   8.09677683e-01,
        -9.84164957e-01,   1.31237473e+00,   4.22860027e+00,
        -2.87015444e+01,   1.19821302e+01,   6.83934060e-01,
         2.84653105e+00,  -2.50642631e-01,   1.54913487e+00,
         4.45441004e+01,  -8.53093490e+02,   8.55568048e+02,
        -4.83703755e+02,   3.93119063e+02,  -3.59932421e+02,
         8.55573343e+02,  -8.54823638e+02,  -4.19256478e+02,
         3.34821448e+02,